In [7]:
# Install necessary libraries
!pip install mediapipe opencv-python-headless gradio

In [8]:
# Import required libraries
import cv2
import mediapipe as mp
import gradio as gr
import numpy as np


In [9]:
# Initialize Mediapipe pose detection
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [13]:
# Function to detect pose and provide feedback
def detect_pose_and_feedback(image):
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize the Mediapipe pose model
    with mp_pose.Pose(static_image_mode=True) as pose:
        # Process the image to detect pose landmarks
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            # Annotate the image with detected landmarks
            annotated_image = image.copy()
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)
            )
 # Feedback logic (example: alignment of shoulders and hips)
            left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value]
            right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value]

            feedback = "Great posture! 🧘 Keep it up!" if (
                abs(left_shoulder.y - right_shoulder.y) < 0.05 and abs(left_hip.y - right_hip.y) < 0.05
            ) else "Adjust your posture: Align your shoulders and hips evenly. ⚠️"

            # Convert BGR to RGB for display
            annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
            return annotated_image_rgb, feedback
        else:
            return image, "No pose landmarks detected. Please try again. ❌"

In [14]:
# Define Gradio Interface
with gr.Blocks() as yoga_app:
    gr.Markdown(
        """
        # 🧘 Yoga Pose Detection & Feedback
        Upload your yoga pose image to receive feedback on your posture!
        This app identifies pose landmarks and suggests alignment corrections for better accuracy. 🌟
        """
    )

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📤 Upload Your Yoga Pose")
            image_input = gr.Image(type="numpy", label="Upload Image")
        with gr.Column():
            gr.Markdown("### 📸 Pose Detection Result")
            image_output = gr.Image(type="numpy", label="Annotated Image")

    feedback_output = gr.Textbox(label="Feedback", lines=3)

    # Button for prediction
    with gr.Row():
        btn = gr.Button("Analyze Pose 🧘‍♂️")

    btn.click(detect_pose_and_feedback, inputs=image_input, outputs=[image_output, feedback_output])

# Launch the Gradio app
yoga_app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d8a78666b882063c5b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
